# Prep

Setting up some prior functionality

In [ ]:
!git clone https://github.com/thepycoder/detr.git

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Initialise ClearML

In [ ]:
# Preferably run this in a terminal if you can, but if in a (colab) notebook, the input is not recognised properly. Press enter once, to give clearml-init an empty input and then fill in the fields one at a time.
!clearml-init

In [ ]:
# Download the data from the clearML server (I just had to run `clearml-data get` to get a local copy of my data)
!wget https://files.community.clear.ml/dragon_detector/dragon_coco.731383ca315447a8bbaf91c4a8bdecf2/artifacts/data/dataset.731383ca315447a8bbaf91c4a8bdecf2.zip
!unzip dataset.731383ca315447a8bbaf91c4a8bdecf2.zip

The rest of the clearml-specific code is added in the `main.py` file and will keep track of every time it is ran from here. Keeping track of all the variables, arguments, metrics, model files and so on.

# Load a model

First we have to decide if our model should be pretrained. 

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning. 

In [ ]:
# Get pretrained weights
checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            map_location='cpu',
            check_hash=True)

# Remove class weights
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]

# SaveOGH
torch.save(checkpoint, 'detr-r50_no-class-head.pth')

# Dataset

Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script

In [ ]:
dataset_file = "dragons" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py

dataDir='/project/data/coco_format' # should lead to a directory with a train and val folder as well as an annotations folder
num_classes = 1 # this int should be the highest ID in your annotations + 1 (for no class) (so here: dragon is class ID 0 and the only one so: 0+1 = 1)

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth"

# Training

We use the main.py script to run our training

In [ ]:
!python main.py \
  --dataset_file $dataset_file \
  --coco_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 10

# Results

Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   